In [304]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [248]:
classificationData = pd.read_csv('./output/classification_events_data.csv')
classificationData

,EventParticipation_Id,EventParticipation_Creator,EventId,EventParticipation_Bidder_UserName,EventParticipation_Bidder_UserId,EventParticipation_SupplierName,EventParticipation_SupplierId,EventParticipation_CommonSupplierName,EventParticipation_CommonSupplierId,EventParticipation_SupplierCity,...,EventParticipation_Owner_UserName,EventParticipation_Owner_UserId,EventParticipation_AcceptedFlag,EventParticipation_DeclinedFlag,EventParticipation_IntendToRespondFlag,EventParticipation_DeclinedToRespondFlag,EventParticipation_ParticipatedFlag,EventParticipation_AwardedFlag,EventParticipation_BidsSubmitted,fraud
0,AAHMAN0V4ez!m,NaN,Doc6737990,Zuo Wei Wu,zwwu,Shang Fa Paper Products Manufactory,sid809,Shang Fa Paper Products Manufactory,sid809,Dongguan,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,1
1,AAHMAN0V4e00S,NaN,Doc6744368,Teri Brown,tbrown,Digi Storage,sid504,Digi Storage,sid504,Boonton,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,False,2,0
2,AAHMAN0V4ezzp,NaN,Doc6737990,Dawn Jackson,djackson,TechStrategy Partners,sid518,TechStrategy Partners,sid518,Stamford,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,False,6,1
3,AAHMAN0V4e0bi,NaN,Doc6738020,Dawn Jackson,djackson,TechStrategy Partners,sid518,TechStrategy Partners,sid518,Stamford,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,False,6,0
4,AAHMAN0V4ez41,NaN,Doc6737990,pseudotaggeds1,pseudotaggeds1,Tokyo Denki K.K.,sid800,Tokyo Denki K.K.,sid800,Chiyoda-ku,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,AAHMAN0V4ifLv,NaN,Doc7001865,fsuser2,fsuser2,Fsupplier2,ACM_44120,Fsupplier2,ACM_44120,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,False,1,1
1808,AAHMAN0V4ifOA,NaN,Doc7001940,Auto Test Supplier 049,autoTestSupplier049,Auto Test Organization 049,ACM_18761,Auto Test Organization 049,ACM_18761,Santa Clara,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,False,1,1
1809,AAHMAN0V4ifDI,NaN,Doc6999590,fsuser1,fsuser1,Fsupplier1,ACM_44114,Fsupplier1,ACM_44114,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,1,1
1810,AAHMAN0V4ifGT,NaN,Doc6999613,fsuser2,fsuser2,Fsupplier2,ACM_44120,Fsupplier2,ACM_44120,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,False,1,1


In [249]:
classificationData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812 entries, 0 to 1811
Data columns (total 23 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   EventParticipation_Id                     1812 non-null   object 
 1   EventParticipation_Creator                0 non-null      float64
 2   EventId                                   1812 non-null   object 
 3   EventParticipation_Bidder_UserName        1812 non-null   object 
 4   EventParticipation_Bidder_UserId          1812 non-null   object 
 5   EventParticipation_SupplierName           1812 non-null   object 
 6   EventParticipation_SupplierId             1812 non-null   object 
 7   EventParticipation_CommonSupplierName     1812 non-null   object 
 8   EventParticipation_CommonSupplierId       1812 non-null   object 
 9   EventParticipation_SupplierCity           1812 non-null   object 
 10  EventParticipation_SupplierState    

In [250]:
awardedEvents = classificationData.query('EventParticipation_AwardedFlag == True')
awardedEvents

,EventParticipation_Id,EventParticipation_Creator,EventId,EventParticipation_Bidder_UserName,EventParticipation_Bidder_UserId,EventParticipation_SupplierName,EventParticipation_SupplierId,EventParticipation_CommonSupplierName,EventParticipation_CommonSupplierId,EventParticipation_SupplierCity,...,EventParticipation_Owner_UserName,EventParticipation_Owner_UserId,EventParticipation_AcceptedFlag,EventParticipation_DeclinedFlag,EventParticipation_IntendToRespondFlag,EventParticipation_DeclinedToRespondFlag,EventParticipation_ParticipatedFlag,EventParticipation_AwardedFlag,EventParticipation_BidsSubmitted,fraud
0,AAHMAN0V4ez!m,NaN,Doc6737990,Zuo Wei Wu,zwwu,Shang Fa Paper Products Manufactory,sid809,Shang Fa Paper Products Manufactory,sid809,Dongguan,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,1
4,AAHMAN0V4ez41,NaN,Doc6737990,pseudotaggeds1,pseudotaggeds1,Tokyo Denki K.K.,sid800,Tokyo Denki K.K.,sid800,Chiyoda-ku,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,1
5,AAHMAN0V4e1Nx,NaN,Doc6756668,Patricia Taylor,ptaylor,Javatec.com,sid516,Javatec.com,sid516,Santa Clara,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,0
6,AAHMAN0V4e0lJ,NaN,Doc6738020,Mao Whei Xiao,mwxiao,China Won Human Resources Int.,sid808,China Won Human Resources Int.,sid808,Guangzhou,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,0
8,AAHMAN0V4ezb!,NaN,Doc6737975,Patricia Taylor,ptaylor,Javatec.com,sid516,Javatec.com,sid516,Santa Clara,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,AAHMAN0V4ifLS,NaN,Doc7001865,fsuser1,fsuser1,Fsupplier1,ACM_44114,Fsupplier1,ACM_44114,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,1,1
1790,AAHMAN0V4ifIk,NaN,Doc7001832,fsuser1,fsuser1,Fsupplier1,ACM_44114,Fsupplier1,ACM_44114,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,1,1
1792,AAHMAN0V4ifO6,NaN,Doc7001940,Samuel Patric,Samuel Patric,A2Z Electronics,ACM_44110,A2Z Electronics,ACM_44110,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,1,1
1798,AAHMAN0V4ifOd,NaN,Doc7001940,Tom Milton,Tom milton,A1 Suppliers Inc.,ACM_44106,A1 Suppliers Inc.,ACM_44106,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,3,1


In [252]:
awardedEvents = awardedEvents.reset_index(drop=True)
awardedEvents

,EventParticipation_Id,EventParticipation_Creator,EventId,EventParticipation_Bidder_UserName,EventParticipation_Bidder_UserId,EventParticipation_SupplierName,EventParticipation_SupplierId,EventParticipation_CommonSupplierName,EventParticipation_CommonSupplierId,EventParticipation_SupplierCity,...,EventParticipation_Owner_UserName,EventParticipation_Owner_UserId,EventParticipation_AcceptedFlag,EventParticipation_DeclinedFlag,EventParticipation_IntendToRespondFlag,EventParticipation_DeclinedToRespondFlag,EventParticipation_ParticipatedFlag,EventParticipation_AwardedFlag,EventParticipation_BidsSubmitted,fraud
0,AAHMAN0V4ez!m,NaN,Doc6737990,Zuo Wei Wu,zwwu,Shang Fa Paper Products Manufactory,sid809,Shang Fa Paper Products Manufactory,sid809,Dongguan,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,1
1,AAHMAN0V4ez41,NaN,Doc6737990,pseudotaggeds1,pseudotaggeds1,Tokyo Denki K.K.,sid800,Tokyo Denki K.K.,sid800,Chiyoda-ku,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,1
2,AAHMAN0V4e1Nx,NaN,Doc6756668,Patricia Taylor,ptaylor,Javatec.com,sid516,Javatec.com,sid516,Santa Clara,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,0
3,AAHMAN0V4e0lJ,NaN,Doc6738020,Mao Whei Xiao,mwxiao,China Won Human Resources Int.,sid808,China Won Human Resources Int.,sid808,Guangzhou,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,0
4,AAHMAN0V4ezb!,NaN,Doc6737975,Patricia Taylor,ptaylor,Javatec.com,sid516,Javatec.com,sid516,Santa Clara,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,AAHMAN0V4ifLS,NaN,Doc7001865,fsuser1,fsuser1,Fsupplier1,ACM_44114,Fsupplier1,ACM_44114,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,1,1
381,AAHMAN0V4ifIk,NaN,Doc7001832,fsuser1,fsuser1,Fsupplier1,ACM_44114,Fsupplier1,ACM_44114,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,1,1
382,AAHMAN0V4ifO6,NaN,Doc7001940,Samuel Patric,Samuel Patric,A2Z Electronics,ACM_44110,A2Z Electronics,ACM_44110,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,1,1
383,AAHMAN0V4ifOd,NaN,Doc7001940,Tom Milton,Tom milton,A1 Suppliers Inc.,ACM_44106,A1 Suppliers Inc.,ACM_44106,Bangalore,...,Customer Support Admin,customersupportadmin,True,False,True,False,True,True,3,1


In [253]:
awardedEvents['fraud'].value_counts()

0    374
1     11
Name: fraud, dtype: int64

In [254]:
awardedEvents = awardedEvents.drop(columns=['EventParticipation_Id', 'EventParticipation_Creator',
       'EventParticipation_Bidder_UserName','EventParticipation_SupplierName',
       'EventParticipation_CommonSupplierName',
       'EventParticipation_CommonSupplierId', 'EventParticipation_SupplierType',
       'EventParticipation_Owner_UserName', 'EventParticipation_Owner_UserId',
       'EventParticipation_AcceptedFlag', 'EventParticipation_DeclinedFlag',
       'EventParticipation_IntendToRespondFlag',
       'EventParticipation_DeclinedToRespondFlag',
       'EventParticipation_ParticipatedFlag',
       'EventParticipation_BidsSubmitted'])
awardedEvents

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud
0,Doc6737990,zwwu,sid809,Dongguan,Guangdong,CN,True,1
1,Doc6737990,pseudotaggeds1,sid800,Chiyoda-ku,Tokyo,JP,True,1
2,Doc6756668,ptaylor,sid516,Santa Clara,CA,US,True,0
3,Doc6738020,mwxiao,sid808,Guangzhou,Guangdong,CN,True,0
4,Doc6737975,ptaylor,sid516,Santa Clara,CA,US,True,0
...,...,...,...,...,...,...,...,...
380,Doc7001865,fsuser1,ACM_44114,Bangalore,NaN,IN,True,1
381,Doc7001832,fsuser1,ACM_44114,Bangalore,NaN,IN,True,1
382,Doc7001940,Samuel Patric,ACM_44110,Bangalore,NaN,IN,True,1
383,Doc7001940,Tom milton,ACM_44106,Bangalore,NaN,IN,True,1


In [255]:
len(awardedEvents.EventId.unique())

106

In [256]:
awardedEvents.info()
awardedEvents.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   EventId                             385 non-null    object
 1   EventParticipation_Bidder_UserId    385 non-null    object
 2   EventParticipation_SupplierId       385 non-null    object
 3   EventParticipation_SupplierCity     385 non-null    object
 4   EventParticipation_SupplierState    375 non-null    object
 5   EventParticipation_SupplierCountry  385 non-null    object
 6   EventParticipation_AwardedFlag      385 non-null    bool  
 7   fraud                               385 non-null    int64 
dtypes: bool(1), int64(1), object(6)
memory usage: 21.6+ KB


EventId                                0
EventParticipation_Bidder_UserId       0
EventParticipation_SupplierId          0
EventParticipation_SupplierCity        0
EventParticipation_SupplierState      10
EventParticipation_SupplierCountry     0
EventParticipation_AwardedFlag         0
fraud                                  0
dtype: int64

In [257]:
awardedEvents['EventParticipation_SupplierState']

0      Guangdong
1          Tokyo
2             CA
3      Guangdong
4             CA
         ...    
380          NaN
381          NaN
382          NaN
383          NaN
384          NaN
Name: EventParticipation_SupplierState, Length: 385, dtype: object

In [258]:
# TODO: Modify Supplier State
awardedEvents.isna().any()

EventId                               False
EventParticipation_Bidder_UserId      False
EventParticipation_SupplierId         False
EventParticipation_SupplierCity       False
EventParticipation_SupplierState       True
EventParticipation_SupplierCountry    False
EventParticipation_AwardedFlag        False
fraud                                 False
dtype: bool

In [259]:
nanvalues = awardedEvents[awardedEvents['EventParticipation_SupplierState'].isnull()]
nanvalues

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud
375,Doc6972453,Samuel Patric,ACM_44110,Bangalore,NaN,IN,True,0
376,Doc6972434,Samuel Patric,ACM_44110,Bangalore,NaN,IN,True,0
377,Doc6972453,Tom milton,ACM_44106,Bangalore,NaN,IN,True,0
378,Doc6998796,Tom milton,ACM_44106,Bangalore,NaN,IN,True,1
379,Doc6999613,fsuser1,ACM_44114,Bangalore,NaN,IN,True,1
380,Doc7001865,fsuser1,ACM_44114,Bangalore,NaN,IN,True,1
381,Doc7001832,fsuser1,ACM_44114,Bangalore,NaN,IN,True,1
382,Doc7001940,Samuel Patric,ACM_44110,Bangalore,NaN,IN,True,1
383,Doc7001940,Tom milton,ACM_44106,Bangalore,NaN,IN,True,1
384,Doc6999590,fsuser1,ACM_44114,Bangalore,NaN,IN,True,1


In [260]:
nanvalues.index

Int64Index([375, 376, 377, 378, 379, 380, 381, 382, 383, 384], dtype='int64')

In [261]:
# Update state
for index in nanvalues.index:
    awardedEvents.at[index, 'EventParticipation_SupplierState'] = 'Karnataka'
    
nanvalues = awardedEvents[awardedEvents['EventParticipation_SupplierState'].isnull()]
nanvalues

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud


In [262]:
# Add Supplier cluster participation 
clusterdEventsData = pd.read_csv('./output/clustered_events_labels.csv')
clusterdEventsData = clusterdEventsData.drop(columns=['labels'], axis=1)
clusterdEventsData

,EventId,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,Doc6759122,6,6,100.00
1,Doc6755796,6,6,100.00
2,Doc6755745,6,6,100.00
3,Doc6757914,6,6,100.00
4,Doc6756787,6,6,100.00
...,...,...,...,...
97,Doc6999590,6,5,83.33
98,Doc7001865,6,6,100.00
99,Doc7001940,6,6,100.00
100,Doc6999613,6,5,83.33


In [263]:
awardedEvents = awardedEvents.merge(clusterdEventsData, how='left', left_on=['EventId'], right_on=['EventId'])
awardedEvents

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,Doc6737990,zwwu,sid809,Dongguan,Guangdong,CN,True,1,6.0,6.0,100.00
1,Doc6737990,pseudotaggeds1,sid800,Chiyoda-ku,Tokyo,JP,True,1,6.0,6.0,100.00
2,Doc6756668,ptaylor,sid516,Santa Clara,CA,US,True,0,6.0,6.0,100.00
3,Doc6738020,mwxiao,sid808,Guangzhou,Guangdong,CN,True,0,6.0,6.0,100.00
4,Doc6737975,ptaylor,sid516,Santa Clara,CA,US,True,0,6.0,6.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...
380,Doc7001865,fsuser1,ACM_44114,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00
381,Doc7001832,fsuser1,ACM_44114,Bangalore,Karnataka,IN,True,1,NaN,NaN,NaN
382,Doc7001940,Samuel Patric,ACM_44110,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00
383,Doc7001940,Tom milton,ACM_44106,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00


In [264]:
awardedEvents.isnull().sum()

EventId                                0
EventParticipation_Bidder_UserId       0
EventParticipation_SupplierId          0
EventParticipation_SupplierCity        0
EventParticipation_SupplierState       0
EventParticipation_SupplierCountry     0
EventParticipation_AwardedFlag         0
fraud                                  0
EventSummary_InvitedSuppliers         16
EventSummary_ParticipSuppliers        16
EventSummary_ParticipationRate        16
dtype: int64

In [265]:
awardedEvents.isna().any()

EventId                               False
EventParticipation_Bidder_UserId      False
EventParticipation_SupplierId         False
EventParticipation_SupplierCity       False
EventParticipation_SupplierState      False
EventParticipation_SupplierCountry    False
EventParticipation_AwardedFlag        False
fraud                                 False
EventSummary_InvitedSuppliers          True
EventSummary_ParticipSuppliers         True
EventSummary_ParticipationRate         True
dtype: bool

In [266]:
nanvalues = awardedEvents[awardedEvents['EventSummary_InvitedSuppliers'].isnull()]
nanvalues

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
105,Doc6757342,pseudotaggeds1,sid800,Chiyoda-ku,Tokyo,JP,True,0,NaN,NaN,NaN
119,Doc6757342,ptaylor,sid516,Santa Clara,CA,US,True,0,NaN,NaN,NaN
125,Doc6757342,zwwu,sid809,Dongguan,Guangdong,CN,True,0,NaN,NaN,NaN
132,Doc6757342,mwxiao,sid808,Guangzhou,Guangdong,CN,True,0,NaN,NaN,NaN
229,Doc6757863,zwwu,sid809,Dongguan,Guangdong,CN,True,0,NaN,NaN,NaN
230,Doc6757863,mwxiao,sid808,Guangzhou,Guangdong,CN,True,0,NaN,NaN,NaN
232,Doc6757863,pseudotaggeds1,sid800,Chiyoda-ku,Tokyo,JP,True,0,NaN,NaN,NaN
242,Doc6757863,ptaylor,sid516,Santa Clara,CA,US,True,0,NaN,NaN,NaN
260,Doc6758401,zwwu,sid809,Dongguan,Guangdong,CN,True,0,NaN,NaN,NaN
268,Doc6758401,mwxiao,sid808,Guangzhou,Guangdong,CN,True,0,NaN,NaN,NaN


In [267]:
fraudEmpty = nanvalues.query('fraud == 1')
fraudEmpty

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
378,Doc6998796,Tom milton,ACM_44106,Bangalore,Karnataka,IN,True,1,NaN,NaN,NaN
381,Doc7001832,fsuser1,ACM_44114,Bangalore,Karnataka,IN,True,1,NaN,NaN,NaN


In [268]:
# Update participation rate
for index in fraudEmpty.index:
    awardedEvents.at[index, 'EventSummary_InvitedSuppliers'] = 68
    awardedEvents.at[index, 'EventSummary_ParticipSuppliers'] = 2
    awardedEvents.at[index, 'EventSummary_ParticipationRate'] = 2.9400
    
nonFraudEmpty = awardedEvents[awardedEvents['EventSummary_InvitedSuppliers'].isnull()]
nonFraudEmpty

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
105,Doc6757342,pseudotaggeds1,sid800,Chiyoda-ku,Tokyo,JP,True,0,NaN,NaN,NaN
119,Doc6757342,ptaylor,sid516,Santa Clara,CA,US,True,0,NaN,NaN,NaN
125,Doc6757342,zwwu,sid809,Dongguan,Guangdong,CN,True,0,NaN,NaN,NaN
132,Doc6757342,mwxiao,sid808,Guangzhou,Guangdong,CN,True,0,NaN,NaN,NaN
229,Doc6757863,zwwu,sid809,Dongguan,Guangdong,CN,True,0,NaN,NaN,NaN
230,Doc6757863,mwxiao,sid808,Guangzhou,Guangdong,CN,True,0,NaN,NaN,NaN
232,Doc6757863,pseudotaggeds1,sid800,Chiyoda-ku,Tokyo,JP,True,0,NaN,NaN,NaN
242,Doc6757863,ptaylor,sid516,Santa Clara,CA,US,True,0,NaN,NaN,NaN
260,Doc6758401,zwwu,sid809,Dongguan,Guangdong,CN,True,0,NaN,NaN,NaN
268,Doc6758401,mwxiao,sid808,Guangzhou,Guangdong,CN,True,0,NaN,NaN,NaN


In [269]:
# Update participation rate
for index in nonFraudEmpty.index:
    awardedEvents.at[index, 'EventSummary_InvitedSuppliers'] = 6
    awardedEvents.at[index, 'EventSummary_ParticipSuppliers'] = 6
    awardedEvents.at[index, 'EventSummary_ParticipationRate'] = 100.00
    
awardedEvents[awardedEvents['EventSummary_InvitedSuppliers'].isnull()]

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate


In [270]:
'''
cols = ['EventParticipation_Bidder_UserId',
       'EventParticipation_SupplierId', 
       'EventParticipation_SupplierCity',
       'EventParticipation_SupplierState',
       'EventParticipation_SupplierCountry', 
       'EventParticipation_AwardedFlag']
#
# Encode labels of multiple columns at once
#
awardedEvents[cols] = awardedEvents[cols].apply(LabelEncoder().fit_transform)
#
# Print head
#
awardedEvents.head()
'''

"\ncols = ['EventParticipation_Bidder_UserId',\n       'EventParticipation_SupplierId', \n       'EventParticipation_SupplierCity',\n       'EventParticipation_SupplierState',\n       'EventParticipation_SupplierCountry', \n       'EventParticipation_AwardedFlag']\n#\n# Encode labels of multiple columns at once\n#\nawardedEvents[cols] = awardedEvents[cols].apply(LabelEncoder().fit_transform)\n#\n# Print head\n#\nawardedEvents.head()\n"

In [271]:
colName = 'EventParticipation_Bidder_UserId'
startKeyIndex = 11

uniqueBidders = awardedEvents[colName].unique()
uniqueBidders

array(['zwwu', 'pseudotaggeds1', 'ptaylor', 'mwxiao', 'tphillips',
       'dstaley', 'sgavaskar', 'ktokuyama', 'Samuel Patric', 'Tom milton',
       'fsuser1'], dtype=object)

In [303]:
# Generate unique keys for each bidders
def updateUniqueKey(awardedEvents, colName, updateRows, startKeyIndex):
    colMap = {}
    colReverseMap = {}
    sequence = startKeyIndex

    for index in range(len(updateRows)):
        uniqueId = updateRows[index]
        colMap[uniqueId] = sequence
        colReverseMap[sequence] = uniqueId
        specificEvent = awardedEvents.query(colName + ' == "' + uniqueId + '"')
        for speicificIndex in specificEvent.index:
            awardedEvents.at[speicificIndex, colName] = sequence
        sequence += 1
    return (colMap, colReverseMap)

In [274]:
results = updateUniqueKey(awardedEvents, colName, uniqueBidders, startKeyIndex)
biddersKey = results[0]
biddersReverseKey = results[1]

awardedEvents

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,Doc6737990,11,sid809,Dongguan,Guangdong,CN,True,1,6.0,6.0,100.00
1,Doc6737990,12,sid800,Chiyoda-ku,Tokyo,JP,True,1,6.0,6.0,100.00
2,Doc6756668,13,sid516,Santa Clara,CA,US,True,0,6.0,6.0,100.00
3,Doc6738020,14,sid808,Guangzhou,Guangdong,CN,True,0,6.0,6.0,100.00
4,Doc6737975,13,sid516,Santa Clara,CA,US,True,0,6.0,6.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...
380,Doc7001865,21,ACM_44114,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00
381,Doc7001832,21,ACM_44114,Bangalore,Karnataka,IN,True,1,68.0,2.0,2.94
382,Doc7001940,19,ACM_44110,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00
383,Doc7001940,20,ACM_44106,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00


In [275]:
colName = 'EventParticipation_SupplierId'
startKeyIndex = 101

uniqueSuppliers = awardedEvents[colName].unique()
uniqueSuppliers

array(['sid809', 'sid800', 'sid516', 'sid808', 'sid501', 'sid503',
       'sid508', 'sid802', 'ACM_44110', 'ACM_44106', 'ACM_44114'],
      dtype=object)

In [276]:
results = updateUniqueKey(awardedEvents, colName, uniqueSuppliers, startKeyIndex)
suppliersKey = results[0]
suppliersReverseKey = results[1]

awardedEvents

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,Doc6737990,11,101,Dongguan,Guangdong,CN,True,1,6.0,6.0,100.00
1,Doc6737990,12,102,Chiyoda-ku,Tokyo,JP,True,1,6.0,6.0,100.00
2,Doc6756668,13,103,Santa Clara,CA,US,True,0,6.0,6.0,100.00
3,Doc6738020,14,104,Guangzhou,Guangdong,CN,True,0,6.0,6.0,100.00
4,Doc6737975,13,103,Santa Clara,CA,US,True,0,6.0,6.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...
380,Doc7001865,21,111,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00
381,Doc7001832,21,111,Bangalore,Karnataka,IN,True,1,68.0,2.0,2.94
382,Doc7001940,19,109,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00
383,Doc7001940,20,110,Bangalore,Karnataka,IN,True,1,6.0,6.0,100.00


In [277]:
colName = 'EventParticipation_SupplierCity'
startKeyIndex = 201

uniqueSupplierCities = awardedEvents[colName].unique()
uniqueSupplierCities

array(['Dongguan', 'Chiyoda-ku', 'Santa Clara', 'Guangzhou', 'Boston',
       'San Jose', 'Sunnyvale', 'Funabashi-shi', 'Bangalore'],
      dtype=object)

In [278]:
results = updateUniqueKey(awardedEvents, colName, uniqueSupplierCities, startKeyIndex)
supplierCitiesKey = results[0]
supplierCitiesReverseKey = results[1]

awardedEvents

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,Doc6737990,11,101,201,Guangdong,CN,True,1,6.0,6.0,100.00
1,Doc6737990,12,102,202,Tokyo,JP,True,1,6.0,6.0,100.00
2,Doc6756668,13,103,203,CA,US,True,0,6.0,6.0,100.00
3,Doc6738020,14,104,204,Guangdong,CN,True,0,6.0,6.0,100.00
4,Doc6737975,13,103,203,CA,US,True,0,6.0,6.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...
380,Doc7001865,21,111,209,Karnataka,IN,True,1,6.0,6.0,100.00
381,Doc7001832,21,111,209,Karnataka,IN,True,1,68.0,2.0,2.94
382,Doc7001940,19,109,209,Karnataka,IN,True,1,6.0,6.0,100.00
383,Doc7001940,20,110,209,Karnataka,IN,True,1,6.0,6.0,100.00


In [279]:
colName = 'EventParticipation_SupplierState'
startKeyIndex = 301

uniqueSupplierStates = awardedEvents[colName].unique()
uniqueSupplierStates

array(['Guangdong', 'Tokyo', 'CA', 'MA', 'Chiba-ken', 'Karnataka'],
      dtype=object)

In [280]:
results = updateUniqueKey(awardedEvents, colName, uniqueSupplierStates, startKeyIndex)
supplierStatesKey = results[0]
supplierStatesReverseKey = results[1]

awardedEvents

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,Doc6737990,11,101,201,301,CN,True,1,6.0,6.0,100.00
1,Doc6737990,12,102,202,302,JP,True,1,6.0,6.0,100.00
2,Doc6756668,13,103,203,303,US,True,0,6.0,6.0,100.00
3,Doc6738020,14,104,204,301,CN,True,0,6.0,6.0,100.00
4,Doc6737975,13,103,203,303,US,True,0,6.0,6.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...
380,Doc7001865,21,111,209,306,IN,True,1,6.0,6.0,100.00
381,Doc7001832,21,111,209,306,IN,True,1,68.0,2.0,2.94
382,Doc7001940,19,109,209,306,IN,True,1,6.0,6.0,100.00
383,Doc7001940,20,110,209,306,IN,True,1,6.0,6.0,100.00


In [281]:
colName = 'EventParticipation_SupplierCountry'
startKeyIndex = 401

uniqueSupplierCountry = awardedEvents[colName].unique()
uniqueSupplierCountry

array(['CN', 'JP', 'US', 'IN'], dtype=object)

In [283]:
results = updateUniqueKey(awardedEvents, colName, uniqueSupplierCountry, startKeyIndex)
supplierCountryKey = results[0]
supplierCountryReverseKey = results[1]

awardedEvents

,EventId,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,fraud,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,Doc6737990,11,101,201,301,401,True,1,6.0,6.0,100.00
1,Doc6737990,12,102,202,302,402,True,1,6.0,6.0,100.00
2,Doc6756668,13,103,203,303,403,True,0,6.0,6.0,100.00
3,Doc6738020,14,104,204,301,401,True,0,6.0,6.0,100.00
4,Doc6737975,13,103,203,303,403,True,0,6.0,6.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...
380,Doc7001865,21,111,209,306,404,True,1,6.0,6.0,100.00
381,Doc7001832,21,111,209,306,404,True,1,68.0,2.0,2.94
382,Doc7001940,19,109,209,306,404,True,1,6.0,6.0,100.00
383,Doc7001940,20,110,209,306,404,True,1,6.0,6.0,100.00


In [284]:
# Do Upsampling
# Define the resampling method
from imblearn.over_sampling import SMOTE

X = awardedEvents.drop(['EventId', 'fraud'],axis=1)
y = awardedEvents.fraud
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

# Apply resampling to the training data only
oversampler = SMOTE(random_state=0)
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

In [285]:
X_resampled

,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,14.0,104.0,204.0,301.0,401.0,True,6.0,6.000000,100.000000
1,14.0,104.0,204.0,301.0,401.0,True,6.0,6.000000,100.000000
2,14.0,104.0,204.0,301.0,401.0,True,6.0,6.000000,100.000000
3,11.0,101.0,201.0,301.0,401.0,True,6.0,6.000000,100.000000
4,21.0,111.0,209.0,306.0,404.0,True,6.0,5.000000,83.330000
...,...,...,...,...,...,...,...,...,...
595,20.107138,110.107138,209.0,306.0,404.0,True,6.0,5.892862,98.214014
596,15.614428,105.614428,205.614428,302.614428,401.968657,True,6.0,6.000000,100.000000
597,14.248754,104.248754,203.887781,302.804863,402.082918,True,6.0,6.000000,100.000000
598,11.903277,101.903277,201.903277,301.903277,401.903277,True,6.0,6.000000,100.000000


In [286]:
'''
testEvents = AwardedEvents.query('EventId == "Doc7001940" | EventId == "Doc7001865"')
testEvents = testEvents.reset_index(drop=True)
cols = ['EventId']
testEvents[cols] = testEvents[cols].apply(LabelEncoder().fit_transform)
testEvents
'''

'\ntestEvents = AwardedEvents.query(\'EventId == "Doc7001940" | EventId == "Doc7001865"\')\ntestEvents = testEvents.reset_index(drop=True)\ncols = [\'EventId\']\ntestEvents[cols] = testEvents[cols].apply(LabelEncoder().fit_transform)\ntestEvents\n'

In [287]:
'''
trainEvents = AwardedEvents.query('EventId != "Doc7001940" & EventId != "Doc7001865"')
trainEvents = trainEvents.reset_index(drop=True)
cols = ['EventId']
trainEvents[cols] = trainEvents[cols].apply(LabelEncoder().fit_transform)
#pd.set_option('display.max_rows', None)
trainEvents
'''

'\ntrainEvents = AwardedEvents.query(\'EventId != "Doc7001940" & EventId != "Doc7001865"\')\ntrainEvents = trainEvents.reset_index(drop=True)\ncols = [\'EventId\']\ntrainEvents[cols] = trainEvents[cols].apply(LabelEncoder().fit_transform)\n#pd.set_option(\'display.max_rows\', None)\ntrainEvents\n'

In [290]:
#X=trainEvents.drop(['fraud'],axis=1)
#y=trainEvents.fraud
model = RandomForestClassifier(random_state=0)
model.fit(X_resampled, y_resampled)

RandomForestClassifier(random_state=0)

In [291]:
X_resampled

,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,14.0,104.0,204.0,301.0,401.0,True,6.0,6.000000,100.000000
1,14.0,104.0,204.0,301.0,401.0,True,6.0,6.000000,100.000000
2,14.0,104.0,204.0,301.0,401.0,True,6.0,6.000000,100.000000
3,11.0,101.0,201.0,301.0,401.0,True,6.0,6.000000,100.000000
4,21.0,111.0,209.0,306.0,404.0,True,6.0,5.000000,83.330000
...,...,...,...,...,...,...,...,...,...
595,20.107138,110.107138,209.0,306.0,404.0,True,6.0,5.892862,98.214014
596,15.614428,105.614428,205.614428,302.614428,401.968657,True,6.0,6.000000,100.000000
597,14.248754,104.248754,203.887781,302.804863,402.082918,True,6.0,6.000000,100.000000
598,11.903277,101.903277,201.903277,301.903277,401.903277,True,6.0,6.000000,100.000000


In [292]:
y_resampled

0      0
1      0
2      0
3      0
4      1
      ..
595    1
596    1
597    1
598    1
599    1
Name: fraud, Length: 600, dtype: int64

In [293]:
X_test

,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
268,14,104,204,301,401,True,6.0,6.0,100.0
250,13,103,203,303,403,True,6.0,6.0,100.0
355,14,104,204,301,401,True,6.0,6.0,100.0
332,14,104,204,301,401,True,6.0,6.0,100.0
56,14,104,204,301,401,True,6.0,6.0,100.0
...,...,...,...,...,...,...,...,...,...
104,13,103,203,303,403,True,6.0,6.0,100.0
167,12,102,202,302,402,True,6.0,6.0,100.0
63,11,101,201,301,401,True,6.0,6.0,100.0
233,14,104,204,301,401,True,6.0,6.0,100.0


In [294]:
y_test

268    0
250    0
355    0
332    0
56     0
      ..
104    0
167    0
63     0
233    0
84     0
Name: fraud, Length: 77, dtype: int64

In [296]:
# Accuracy for Training Events
y_train_pred = model.predict(X_resampled)
accuracy_score(y_resampled, y_train_pred)

0.995

In [297]:
# Accuracy for Testing Events
y_test_pred = model.predict(X_test)
accuracy_score(y_test, y_test_pred)

0.974025974025974

In [298]:
classificationData.query('fraud == 1')['EventId'].unique()

array(['Doc6737990', 'Doc6998816', 'Doc6998796', 'Doc6999613',
       'Doc7001865', 'Doc7001940', 'Doc7001832', 'Doc6999590'],
      dtype=object)

In [299]:
'''
Doc6744368
Doc6762337
Doc6757695
Doc6853094
Doc6871841
Doc6972434
Doc6900882
Doc6972453
'''
data = classificationData.query("EventId in ('Doc6744368', 'Doc6762337', 'Doc6757695', 'Doc6853094', 'Doc6871841', 'Doc6972434',  'Doc6900882',  'Doc6972453')")

In [300]:
data[['EventId', 'fraud']]

,EventId,fraud
1,Doc6744368,0
12,Doc6744368,0
15,Doc6744368,0
24,Doc6744368,0
34,Doc6744368,0
...,...,...
1768,Doc6972453,0
1769,Doc6972453,0
1770,Doc6972453,0
1771,Doc6972453,0


In [302]:
bidder = biddersKey['pseudotaggeds1']
supplier = suppliersKey['sid800']
supplierCity = supplierCitiesKey['Chiyoda-ku']
supplierState = supplierStatesKey['Tokyo']
supplierCountry = supplierCountryKey['JP']
invitedSuppliers = 6.0
participSuppliers = 6.0
participationRate = (1 - ((invitedSuppliers - participSuppliers)/invitedSuppliers)) * 100


newSample = [[bidder, supplier, supplierCity, supplierState, supplierCountry, True, invitedSuppliers, participSuppliers, participationRate]]
y_new_sample_pred = model.predict(newSample)

y_new_sample_pred

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])

In [309]:
input = [['pseudotaggeds1', 'sid800', 'Chiyoda-ku', 'Tokyo', 'JP', True, 6.0, 6.0, 100.00]]
inputDf = pd.DataFrame(input, columns=['EventParticipation_Bidder_UserId',
                                 'EventParticipation_SupplierId', 
                                 'EventParticipation_SupplierCity',
                                 'EventParticipation_SupplierState',
                                 'EventParticipation_SupplierCountry',	
                                 'EventParticipation_AwardedFlag',	
                                 'EventSummary_InvitedSuppliers',	
                                 'EventSummary_ParticipSuppliers',
                                 'EventSummary_ParticipationRate'])
inputDf


,EventParticipation_Bidder_UserId,EventParticipation_SupplierId,EventParticipation_SupplierCity,EventParticipation_SupplierState,EventParticipation_SupplierCountry,EventParticipation_AwardedFlag,EventSummary_InvitedSuppliers,EventSummary_ParticipSuppliers,EventSummary_ParticipationRate
0,pseudotaggeds1,sid800,Chiyoda-ku,Tokyo,JP,True,6.0,6.0,100.0


In [311]:
inputDf['EventParticipation_Bidder_UserId'][0]

'pseudotaggeds1'

In [314]:
bidder = biddersKey[inputDf['EventParticipation_Bidder_UserId'][0]]
supplier = suppliersKey[inputDf['EventParticipation_SupplierId'][0]]
supplierCity = supplierCitiesKey[inputDf['EventParticipation_SupplierCity'][0]]
supplierState = supplierStatesKey[inputDf['EventParticipation_SupplierState'][0]]
supplierCountry = supplierCountryKey[inputDf['EventParticipation_SupplierCountry'][0]]
awardedFlag = inputDf['EventParticipation_AwardedFlag'][0]
invitedSuppliers = inputDf['EventSummary_InvitedSuppliers'][0]
participSuppliers = inputDf['EventSummary_ParticipSuppliers'][0]
participationRate = inputDf['EventSummary_ParticipationRate'][0]
newModSample = [[bidder, supplier, supplierCity, supplierState, supplierCountry, awardedFlag, invitedSuppliers, participSuppliers, participationRate]]

y_new_sample_pred = model.predict(newModSample)
y_new_sample_pred

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])